# <center> Hunt for Exoplanets using Machine Learning

--- 
    
**Project ID: HEML0922**

**Project Name: Hunting for Exoplanet with Machine Learning**
    
---

<center> <img src = "https://c.tenor.com/3zb7e2Hb9f4AAAAC/galaxy-stars.gif" width = 100%>

#### **Let us start by [understanding exoplanets](https://docs.google.com/presentation/d/1-zjKAaiDt3gs4NkHbWsT4AzgZ-jqL6Oz/edit?usp=sharing&ouid=105423306669124151882&rtpof=true&sd=true)!**


### In this notebook we will look into the following:-          
**1)** <a href='#section1'>Explore the Exoplanet Dataset</a>          
**2)** <a href='#section2'>Handling outliers</a>        
**3)** <a href='#section3'>Understand KNN model for classification</a>   
**4)** <a href='#section4'>Implementing KNN without Balancing the data</a>          
**5)** <a href='#section5'>Handling imbalance data and then implementing KNN</a>         
**6)** <a href='#section6'>Task for you</a> 

<a id='section1'></a>

### Explore the Exoplanet Dataset 


We will be dealing with the **[Kepler Space Telescope data](https://www.kaggle.com/datasets/keplersmachines/kepler-labelled-time-series-data)**      

<img src="https://i.ytimg.com/vi/3yij1rJOefM/maxresdefault.jpg" width = 500 height = 300>        

**Steps to add the data in Kaggle notebook:-**           
**1)** Click on top right arrow of the notebook      
**2)** Click on "Add Data"         
**3)** Copy the link of "dataset" and paste it in search      
**4)** Click on "+" sign to add this dataset to the notebook

#### Importing the most needed libraries

In [98]:
#********************************************
import pandas as pd
import seaborn as sns
#********************************************
import numpy as np 
import matplotlib.pyplot as plt
#********************************************
import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
#********************************************

#### Getting into the data

In [163]:
# Let us begin with Train data
train_df = pd.read_csv('../input/kepler-labelled-time-series-data/exoTrain.csv')
train_df.head(10)

In [131]:
# Check the shape of train data
train_df.shape

> *We can understand this data based on the transit method for detecting exoplanets.*  

> *There are total of 5087 stars in this data.*

> *For each star, we have 3197 flux values at different time intervals.*

> *These flux values are used to plot the light curves we saw earlier to detect if a star has exoplanet(s) orbiting it.*

#### Check for Missing Values

In [172]:
# Display the rows with null values
train_df[train_df.isnull().any(axis = 1)]  # axis = 1 ---> column

> *There are **no missing values**! We can also visualise it through heatmap.*

In [102]:
sns.heatmap(train_df.isnull(), cmap = 'Set2', cbar = False)

> *The horizontal dashes in this plot would indicate the presence of missing values in respective column.*   

> *As there aren't any of such dashes seen we can conclude that there are no missing values in this data.*

#### Decoding labels in the data

In [103]:
# Check how many labels are there
train_df['LABEL'].unique()

In [104]:
# Extract the index for the stars labelled as 2
idx_lab2 = list(train_df[train_df['LABEL'] == 2].index)
print(f"Index list for label 2 star in the data:-\n{idx_lab2}\n")

> *There are total of **two classes**; one is for stars with exoplanets and the other for stars without exoplanets*

> *Very few index for label 2 indicates that this class must belong to stars with exoplanets*

> *We can also visualise this using countplot*

In [105]:
# Visualise these values using countplot
plt.figure(figsize = (3, 5))                                                   
ax = sns.countplot('LABEL', data = train_df, palette = 'Set2')                    
ax.bar_label(ax.containers[0])
plt.title("Visualising count of classes\n1 ~ Non Exoplanets | 2 ~ Exoplanets\n", 
          fontsize = 15, color = 'red', weight = 'bold')
plt.show()

> *There is a **huge imbalance** in the data which isn't good for KNN (explained later in this notebook).*

> *We will need to balance it using some resampling technique and we will use RandomOverSampler for this data.*

> *We'll do that after building the model with imbalanced dataset to compare the results!*

#### Replacing the labels
For ease of our model its always better to feed in the data in terms of 0 and 1         
- Stars with Exoplanets: 2 $\rightarrow$ 1      
- Stars without Exoplanets: 1 $\rightarrow$ 0

In [132]:
# Replacing labels 
train_df = train_df.replace({'LABEL' : {1:0, 2:1}})
print("Replacing labels...")

# Check the labels now
print("Done!\n")
uniq_val = train_df.LABEL.unique()
print(f"There are {len(uniq_val)} classes in the data:-")
print(f"{uniq_val[0]} - Stars with Exoplanets\n{uniq_val[1]} - Stars without Exoplantes")

#### Visualising the light curves in this data
When a planet passes between an observer and the star, the flux value decreases and hence we see a dip in light curves with exoplanets


<img src = "https://media3.giphy.com/media/495uiXmwMRnWvXU9gm/giphy.gif" width = 100%>

In [133]:
# Drop label column to plot only the flux values
plot_df = train_df.drop(['LABEL'], axis = 1)

# X - axis data: Replace FLUX. from each column names
col_names = list(plot_df.columns)
time = [int(flux_prefix.replace("FLUX.", "")) for flux_prefix in col_names]

# Function to plot flux variation of star
def flux_plot(df, candidate, exo = True):
    color = 'b' if exo == True else 'm'
    plt.figure(figsize=(15, 5))
    plt.plot(time, df.iloc[candidate-1], linewidth = .5, color = color)
    title1, clr1 = f"Flux Variation of star {candidate} with Exoplanents", 'olive'
    title2, clr2 = f"Flux Variation of star {candidate} without Exoplanets", 'tab:red'
    plt.title(title1, color = clr1) if exo == True else plt.title(title2, color = clr2)
    plt.xlabel("Time")
    plt.ylabel("Flux Variation")

In [134]:
# Example of light curves
exo, n_exo = [4, 14, 34], [99, 199, 2999]

for candidate in range(len(exo)):
    flux_plot(plot_df, exo[candidate], exo = True)
    flux_plot(plot_df, n_exo[candidate], exo = False)

<a id="section2"></a>

### Extreme outliers 
- We can see random **huge spikes** especially in stars without exoplanets which can be considered as extreme outliers

- KNN can be sensitive to outliers (explained later in this notebook) so we will need to handle it 

- We can also visualise these extreme outliers through boxplot

In [142]:
# Boxplot to visualise outliers
plt.figure(figsize = (20, 9))
plt.suptitle("Box Plot to visualise outliers", ha = 'right', color = 'red', weight = 'bold')
for i in range(1, 4):
    plt.subplot(1, 4, i)
    sns.boxplot(data=train_df, x='LABEL', y = 'FLUX.' + str(i))
    plt.xlabel("")
    plt.ylabel("")
    plt.title("FLUX " + str(i) + "\n", color = 'b', fontsize = 13)


> *We can see that the flux values more than $0.25 x 10^6$ are extreme outliers.*        

> *We can either drop it or replace its value with upper bridge value. For this usecase, we will simply drop it.*     

> *However you can try to compute on your own the upper bridge value using the formula given below:-*   

> $UB = Q3 + 3 \times IQR$; **UB** - upper bridge, **Q3** - 75th percentile, **IQR** - Interquartile range

In [167]:
# Get the extreme outliers
extreme_outliers = train_df[train_df['FLUX.2'] > 0.25e6]
extreme_outliers

In [168]:
# Drop the extreme outlier
print("Droping Extreme Outliers...")
train_df.drop(extreme_outliers.index, axis = 0, inplace = True)  # axis = 0 ----> row
print("Done!")

> *We have dropped the star at location 3340 which contributed to extreme outliers, let us visualise the box plots again*

In [173]:
# Cross check via any random box plot
sns.boxplot(data=train_df, x='LABEL', y = 'FLUX.' + str(np.random.randint(1000)))

<a id = 'section3'></a>

### Understanding K - Nearest Neighbors (KNN) Algorithm for Classification Tasks

#### Why choose KNN for this task?
- In this dataset we saw that there were outliers + it was imbalanced       
- KNN is **sensitive** to both **outliers and imbalnced data** which you will understand in a while
- Hence, we choose this model to demonstrate how to handle outliers and imbalanced dataset
- Moreover KNN is one of the simplest ML algorithms based on Supervised Learning technique
- KNN are widely used for classification (binary and multiclass classification) as compared to regression tasks

#### How does KNN work?     

**Steps:-**      
**1)** Select the **number of K neighbors** for the new data point      
**2)** Calculate the <a href = '#euclidean'>**Euclidean distance**</a> from this point to the other points in the data     
**3)** Take the **K nearest neighbors** as per the calculated Euclidean distance     
**4)** Among these neighbors, **count** the number of data points of each category       
**5)** The new data point belongs to the **cateogory with maximum data points available**   

*Here is one demonstration to understand how this algorith works:-*

<img src = "https://machinelearningknowledge.ai/wp-content/uploads/2018/08/KNN-Classification.gif" width = 500 height = 300>

<a id='euclidean'></a>


**Euclidean Distance** *between two points is simply calculated using the distance between two points formula in a cartesian coordinate system:-*      

<img src = "https://hlab.stanford.edu/brian/making7.gif" width = 400 height = 300>

#### How to select value for K in this algorithm      
- We need to try out some values of K and figure out which one is working best out of them      
- Usually the preferred value of K is 5      
- A very low value of K (K = 1, K = 2) can lead to effects of outliers in the model
- A very high value of K can lead to the biasness towards imbalanced dataset

<img src = "https://drive.google.com/uc?id=1t1l1vqxIn9sNwyFWbI4NVxyeXC_-GQVu" width = 400 height = 300>  <img src = "https://drive.google.com/uc?id=1vmctmaafTVxa4mVt6raExJ-fEL3VBdde" width = 400 height = 300>

- If the value of K isn't very low and dataset is balanced than the outliers would not really effect our model
- Lets take K = 150 in that case you can see the imbalance in the data can lead to a very poor result! 


#### Advantages of KNN
**1)** One of the simplest to understand and implement        
**2)** Depending on value of K it can be robust to the noisy training data         
**3)** It can be more effective if training data is large          

#### Disadvantages of KNN         
**1)** Sometimes detrming the value of K can become a complex task           
**2)** High computation cost as we are calculating the distances between the data points for all the training data

<a id='section4'></a>

### Implementing KNN after handling the extreme outliers but have yet not balanced the data
*It would be interesting to compare the results with and without imbalance in our data. Let us first start with imbalanced data:-*

In [138]:
# Extract dependent and independent features
x = train_df.drop(['LABEL'], axis = 1)
y = train_df.LABEL

print(f"Take a look over ~\n\nX train array:-\n{x.values}\n\nY train array:-\n{y.values}")

In [113]:
# Splitting this dataset into training and testing set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 0)

In [114]:
# Feature scaling
from sklearn.preprocessing import StandardScaler 

sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

# Checking the minimum, mean and maxmum value after scaling
print("X_train after scaling ~\n")
print(f"Minimum:- {round(np.min(X_train_sc),2)}\nMean:- {round(np.mean(X_train_sc),2)}\nMax:- {round(np.max(X_train_sc), 2)}\n")
print("--------------------------------\n")
print("X_test after scaling ~\n")
print(f"Minimum:- {round(np.min(X_test_sc),2)}\nMean:- {round(np.mean(X_test_sc),2)}\nMax:- {round(np.max(X_test_sc), 2)}\n")

In [115]:
# Fiting the KNN Classifier Model on to the training data
from sklearn.neighbors import KNeighborsClassifier as KNC

# Choosing K = 10
knn_classifier = KNC(n_neighbors=5,metric='minkowski',p=2)  
'''metric is to be by default minkowski for p = 2 to calculate the Eucledian distances'''

# Fit the model
knn_classifier.fit(X_train_sc, y_train)

# Predict
y_pred = knn_classifier.predict(X_test_sc)

# Results
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_curve, auc

print('\nValidation accuracy of KNN is', accuracy_score(y_test,y_pred))
print("\n-------------------------------------------------------")
print ("\nClassification report :\n",(classification_report(y_test,y_pred)))

#Confusion matrix
plt.figure(figsize=(15,11))
plt.subplots_adjust(wspace = 0.3)
plt.suptitle("KNN Performance before handling the imbalance in the data", color = 'r', weight = 'bold')
plt.subplot(221)
sns.heatmap(confusion_matrix(y_test,y_pred),annot=True,cmap="Set2",fmt = "d",linewidths=3, cbar = False,
           xticklabels=['nexo', 'exo'], yticklabels=['nexo','exo'], square = True)
plt.xlabel("True Labels", fontsize = 15, weight = 'bold', color = 'tab:pink')
plt.ylabel("Predicited Labels", fontsize = 15, weight = 'bold', color = 'tab:pink')
plt.title("CONFUSION MATRIX",fontsize=20, color = 'm')

#ROC curve and Area under the curve plotting
predicting_probabilites = knn_classifier.predict_proba(X_test_sc)[:,1]
fpr,tpr,thresholds = roc_curve(y_test,predicting_probabilites)
plt.subplot(222)
plt.plot(fpr,tpr,label = ("AUC :",auc(fpr,tpr)),color = "g")
plt.plot([1,0],[1,0],"k--")
plt.legend()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title("ROC - CURVE & AREA UNDER CURVE",fontsize=20, color = 'm')
plt.show()



> *Even though the accuracy is amazing, this isn't really a good model. This is due to the huge imbalance in the dataset!*      

> *We need to check for other metrics like **precission**, **recall**, **f1 score** in such models*       

<center> <img src = "https://www.researchgate.net/profile/B-Aksasse/publication/326866871/figure/fig3/AS:669601385959430@1536656819610/22-confusion-matrix-and-associated-measures.ppm" width = 700 height = 300>               
    
    
> *Beliving this as a very good model based on only the accuracy can really perform bad on an unseen data*

### Handling the imbalance in the data and then applying KNN
*There are many techniques available out of which we will be trying* ***RandomOverSampler***:-     
 RandomOverSampler is over-sampling by duplicating some of the original samples of the minority class

<a id='section5'></a>

In [175]:
# Handling imbalanced data using RandomOverSampler
from imblearn.over_sampling import RandomOverSampler
from collections import Counter

ros = RandomOverSampler()
x_ros, y_ros = ros.fit_resample(x, y)  # Taking the original x, y as arguments

print(f"Before sampling:- {Counter(y)}")
print(f"After sampling:- {Counter(y_ros)}")

In [216]:
# Visualise it
y_ros.value_counts().plot(kind='bar', title='After aplying RandomOverSampler');


#### Repeating the above steps

In [119]:
#  ****************************************************************
# | Performing split and scaling on the random over sampled data  |
# ****************************************************************

X_train, X_test, y_train, y_test = train_test_split(x_ros, y_ros, test_size = 0.3, random_state = 0)

sc = StandardScaler()
X_train_sc = sc.fit_transform(X_train)
X_test_sc = sc.transform(X_test)

##### Creating a function to try to fetch the optimal value of K

In [120]:
# Create function to fetch the optimal value of K
def optimal_Kval_KNN(start_k, end_k, x_train, x_test, y_train, y_test, progress = True):
    ''' 
    This function takes in the following arguments -
    start_k - start value of k
    end_k - end value of k
    x_train - independent training values for training the KNN
    x_test - independent testing values for prediction
    y_train - dependent training values for training KNN
    y_test - dependent testing values for computing error rate
    progress - if true shows the progress for each k (by default its set to True)
    '''
    # Header
    print(f"Fetching the optimal value of K in between {start_k} & {end_k} ~\n\nIn progress...")
    
    # Empty list to append error rate
    mean_err = []
    for K in range(start_k, end_k + 1):                         # Generates K from start to end-1 values
        knn = KNC(n_neighbors = K)                              # Build KNN for respective K value
        knn.fit(x_train, y_train)                               # Train the model
        err_rate = np.mean(knn.predict(x_test) != y_test)       # Get the error rate
        mean_err.append(err_rate)                               # Append it
        # If progress is true display the error rate for each K
        if progress == True:print(f'For K = {K}, mean error = {err_rate:.3}')
        
    # Get the optimal value of k and corresponding value of mean error
    k, val = mean_err.index(min(mean_err))+1, min(mean_err)
    
    # Footer
    print('\nDone! Here is how error rate varies wrt to K values:- \n')
    
    # Display how error rate changes wrt K values and mark the optimal K value
    plt.figure(figsize = (5,5))
    plt.plot(range(start_k,end_k + 1), mean_err, 'mo--', markersize = 8, markerfacecolor = 'c',
            linewidth = 1)          # plots all mean error wrt K values
    plt.plot(k, val, marker = 'o', markersize = 8, markerfacecolor = 'gold', 
             markeredgecolor = 'g') # highlits the optimal K
    plt.title(f"The optimal performance is obtained at K = {k}", color = 'r', weight = 'bold',
             fontsize = 15)
    plt.ylabel("Error Rate", color = 'olive', fontsize = 13)
    plt.xlabel("K values", color = 'olive', fontsize = 13)
    
    '''returns the optimal value of k'''
    return k

In [121]:
k = optimal_Kval_KNN(1, 10, X_train_sc, X_test_sc, y_train, y_test)

> *Seems like **K = 1** shall do the job! Let's try*

In [260]:
# Fiting the KNN Classifier Model on to the training data after

# Choosing K = 1
knn_classifier = KNC(n_neighbors=1,metric='minkowski',p=2)  
'''metric is to be by default minkowski for p = 2 to calculate the Eucledian distances'''

# Fit the model
knn_classifier.fit(X_train_sc, y_train)

# Predict
y_pred = knn_classifier.predict(X_test_sc)

# Results
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_curve, auc

print('\nValidation accuracy of KNN is', accuracy_score(y_test,y_pred))
print("\n-------------------------------------------------------")
print ("\nClassification report :\n",(classification_report(y_test,y_pred)))

#Confusion matrix
plt.figure(figsize=(15,11))
plt.subplots_adjust(wspace = 0.3)
plt.suptitle("KNN Performance after handling the imbalance in the data", color = 'b', weight = 'bold')
plt.subplot(221)
sns.heatmap(confusion_matrix(y_test,y_pred),annot=True,cmap="Set2",fmt = "d",linewidths=3, cbar = False,
           xticklabels=['nexo', 'exo'], yticklabels=['nexo','exo'], square = True)
plt.xlabel("True Labels", fontsize = 15, weight = 'bold', color = 'm')
plt.ylabel("Predicited Labels", fontsize = 15, weight = 'bold', color = 'm')
plt.title("CONFUSION MATRIX",fontsize=20, color = 'purple')

#ROC curve and Area under the curve plotting
predicting_probabilites = knn_classifier.predict_proba(X_test_sc)[:,1]
fpr,tpr,thresholds = roc_curve(y_test,predicting_probabilites)
plt.subplot(222)
plt.plot(fpr,tpr,label = ("AUC :",auc(fpr,tpr)),color = "g")
plt.plot([1,0],[1,0], 'k--')
plt.legend(loc = "best")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title("ROC - CURVE & AREA UNDER CURVE",fontsize=20, color = 'm')
plt.show()

> *We can see now all the metrics we talked about earlier is showing good results on the splitted testing set!*

<a id='section6'></a>

### Task for you

- Is this model working well for unseen data (test set)? Try it yourself! 
- If **yes**,
    - *Can you name any other models that would work better than KNN?*
    - *Try building a model that would work better than what you get after testing the unseen data*
    - [*Submit*](https://forms.gle/gVW3Spv148dMzamo9) *your notebook by completing these tasks and get certified for solving this problem*
-If **not**, 
    - *What could be the reason we might have overlooked while building our model?*
    - *Can you come up with a better model (not necessary to use KNN) that will work well with the test set?* 
    - [*Submit*](https://forms.gle/gVW3Spv148dMzamo9) *your notebook by completing these tasks and get certified for solving this problem*


---

# <center> THE END